In [1]:
import numpy as np
from sklearn.utils import shuffle

In [4]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
def diff_sigmoid(x):
    return sigmoid(x)*(1-sigmoid(x))

def softmax(x):
    return np.exp(x)/np.sum(np.exp(x),axis=1)[:,np.newaxis]
def diff_softmax(x):
    return softmax(x)*(np.ones(x.shape)-softmax(x))

def tanh(x):
    return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
def diff_tanh(x):
    return 1-tanh(x)**2

In [3]:
#ネットワーク全体の順伝播
def fprops(layers, x):
    z = x
    for layer in layers:
        z = layer.fprop(z)
    return z

#ネットワーク全体の誤差逆伝播
def bprops(layers, delta):
    for i,layer in enumerate(layers[::-1]):
        if i==0:
            layer.delta = delta
        else:
            delta = layer.bprop(delta,_W)
        _W = layer.W

In [5]:
#train関数とtest関数
def train(X,d,eps=1):
    #forward propagation
    y = fprops(layers,X)
        
    #cost function & delta
    cost = np.sum(-d*np.log(y)-(1-d)*np.log(1-y))
    delta = y-d
    
    #back propagation
    bprops(layers,delta)

    #update parameters
    z = X
    for layer in layers:
        dW = np.dot(z.T,layer.delta)
        db = np.dot(np.ones(len(z)),layer.delta)

        layer.W = layer.W - eps*dW
        layer.b = layer.b - eps*db

        z = layer.fprop(z)
        
    #train cost
    y = fprops(layers,X)
    cost = np.sum(-d*np.log(y)-(1-d)*np.log(1-y))
    
    return cost

def test(X,d):
    #test cost
    y = fprops(layers,X)
    cost = np.sum(-d*np.log(y)-(1-d)*np.log(1-y))
    return cost,y

In [15]:
# 出力yはone-of-k表現
# 最終層の活性化関数はsoftmax関数，誤差関数は多クラス交差エントロピー
# 最終層のデルタは教科書参照

import matplotlib.pyplot as plt
import numpy

from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelBinarizer

from sklearn.datasets import fetch_mldata

mnist = fetch_mldata('MNIST original')

X, y = shuffle(mnist.data, mnist.target)
X = X / 255.0
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)

# 教師信号の数字を1-of-K表記に変換
labels_train = LabelBinarizer().fit_transform(train_y)
labels_test = LabelBinarizer().fit_transform(test_y)

layers = [Layer(28*28,100,tanh,diff_tanh),
          Layer(100,10,softmax,diff_softmax)]

#パラメータの更新 学習
#epoch
#mlp.fit(X_train, labels_train, learning_rate=0.01, epochs=100000)
for epoch in range(10):
    print epoch
    #online learning
    train_X, train_y = shuffle(train_X, train_y)
    for x,y in zip(train_X,train_y):
        train(x[np.newaxis,:],y)
    cost,pred_y = test(test_X,test_y)
print pred_y

0


/usr/local/pyenv/versions/anaconda-2.4.0/lib/python2.7/site-packages/ipykernel/__main__.py:12: RuntimeWarning: overflow encountered in exp
/usr/local/pyenv/versions/anaconda-2.4.0/lib/python2.7/site-packages/ipykernel/__main__.py:12: RuntimeWarning: invalid value encountered in divide


ValueError: operands could not be broadcast together with shapes (14000,) (14000,10) 